In [6]:
#!pip install torch==1.8.1 torchvision==0.4.1

In [7]:
#from google.colab import drive
#drive.mount('/content/drive')

In [8]:
import numpy as np
import os
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
torch.cuda.is_available()

True

In [9]:
def getGaussian(Nfft, Pk):
    xf = np.random.normal(0,1,Nfft)+1j*np.random.normal(0,1,Nfft)
    xf *= Pk
    return np.fft.irfft(xf)

def getNonGaussian(t):
    # Signal with non-Gaussian shape
    freq = np.random.uniform(200,500)
    phase = np.random.uniform(0,2*np.pi)
    sigma = 0.03
    pos = np.random.uniform(3*sigma,1-3*sigma)
    ampl = np.random.uniform (0.1,0.2)
    rfi = ampl*np.cos(phase+freq*t)*np.exp(-(t-pos)**2/(2*sigma**2))
    return rfi

def Gaussianize(signal):
    fsig = np.fft.rfft(signal)
    rot = np.exp(1j*np.random.uniform(0,2*np.pi,len(fsig)))
    return np.fft.irfft((fsig*rot))  

In [10]:
# define the network classes
# the decoder network
class Decoder(nn.Module):
  def __init__(self, z_dim, hidden_dim, out_dim):
    super(Decoder, self).__init__()
    self.main = nn.Sequential(
      nn.Linear(z_dim, hidden_dim),
      nn.LeakyReLU(0.02, inplace=False),

      nn.Linear(hidden_dim, hidden_dim * 2),
      nn.LeakyReLU(0.02, inplace=False),

      nn.Linear(hidden_dim * 2, out_dim, bias=False),
    )
    
  def forward(self, x):
    out = self.main(x)
    return out
    #return x-self.main(x)



# the encoder network
class Encoder(nn.Module):
  def __init__(self, input_dim, hidden_dim, z_dim):
    super(Encoder, self).__init__()

    self.main = nn.Sequential(
      nn.Linear(input_dim, hidden_dim * 2),
      nn.LeakyReLU(0.02, inplace=False),

      nn.Linear(hidden_dim * 2, hidden_dim),
      nn.LeakyReLU(0.02, inplace=False),
      nn.Dropout(0.2),

      nn.Linear(hidden_dim, z_dim)
    )
    
  def forward(self, x):
    out = self.main(x)
    return out
    #return x-self.main(x)

In [11]:
# training constants
BATCH_SIZE = 32
Z_DIM = 16
HIDDEN_DIM = 256
LR = 0.0002
EPOCHS = 30

# data parameters
N = 1024
Nfft = N//2+1
k = np.linspace(0,1,Nfft)
t = np.linspace(0,1,N)
Pk = (1+np.exp(-(k-0.5)**2/(2*0.1**2)))*np.exp(-k/0.5)

# train data
g_train_array = torch.stack([torch.from_numpy(getGaussian(Nfft, Pk)) for i in range(100000)])
ng_train_array = torch.stack([torch.from_numpy(getNonGaussian(t)) for i in range(100000)])

# test data
g_test_array = torch.stack([torch.from_numpy(getGaussian(Nfft, Pk)) for i in range(10)])
ng_test_array = torch.stack([torch.from_numpy(getNonGaussian(t)) for i in range(10)])

# create dataloaders
g_trainloader = DataLoader(
      torch.utils.data.TensorDataset(g_train_array),
      batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True, drop_last=True
)
ng_trainloader = DataLoader(
      torch.utils.data.TensorDataset(ng_train_array),
      batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True, drop_last=True
)

# initialize the networks
netD = Decoder(z_dim=Z_DIM, hidden_dim=HIDDEN_DIM, out_dim=N).cuda()
netE = Encoder(input_dim=N, hidden_dim=HIDDEN_DIM, z_dim=Z_DIM).cuda()

# define the optimizers
optimizer = optim.Adam([{'params': netE.parameters()}, 
                        {'params': netD.parameters()}], lr=LR, betas=(0.5, 0.999))

# loss criterion
recons_criterion = nn.MSELoss()

# training loop
iters = 0

for epoch in range(EPOCHS):
  # iterate through the dataloaders
  for i, (g, ng) in enumerate(zip(g_trainloader, ng_trainloader)):
      # set to train mode
      netE.train()
      netD.train()

      ng = ng[0].float().cuda()
      g = g[0].float().cuda()
      bs = ng.shape[0]

      # new inputs
      sigs = g + ng
      gaussianized = torch.stack([torch.from_numpy(Gaussianize(sig.cpu().numpy())) for sig in sigs]).cuda()
      modsig = (sigs + gaussianized).float()
    
      # for the noisy input get latent representation from the encoder
      z_out = netE(modsig)

      # decode the latent representation
      recons_out = netD(z_out)

      # compute the loss
      loss = recons_criterion(recons_out, ng)

      # backpropagate and update the weights
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      # print the training losses
      if iters % 100 == 0:
        print('[%3d/%d][%3d/%d]\tLoss: %.4f' 
            % (epoch, EPOCHS, i, len(g_trainloader), loss.item()))
          
      # visualize generated samples
      if iters % 2000 == 0:
        # set to eval mode
        netD.eval()
        netE.eval()

        with torch.no_grad():
          # new inputs
          sigs = g_test_array.float().cuda() + ng_test_array.float().cuda()
          gaussianized = torch.stack([torch.from_numpy(Gaussianize(sig.cpu().numpy())) for sig in sigs]).cuda()
          modsig = (sigs + gaussianized).float()
          z_out = netE(modsig)
          recons_out = netD(z_out)

        # display a random sample from the test set
        rand_int = np.random.randint(10)

        # plot/save generated samples
        plt.figure(figsize=(17,10))
        ax = plt.subplot(2,3,1)
        plt.plot(t, ng_test_array[rand_int].cpu().numpy()) # ng
        ax.set_title("ng")

        ax = plt.subplot(2,3,2)
        plt.plot(t, g_test_array[rand_int].cpu().numpy()) # g
        ax.set_title("g")

        ax = plt.subplot(2,3,3)
        plt.plot(t, (g_test_array[rand_int].cpu() + ng_test_array[rand_int].cpu()).numpy()) # g + ng 
        ax.set_title("sig = ng + g")

        ax = plt.subplot(2,3,4)
        plt.plot(t, gaussianized[rand_int].cpu().numpy()) # gaussianize(ng + g)
        ax.set_title("Gauss(sig)")

        ax = plt.subplot(2,3,5)
        plt.plot(t, modsig[rand_int].cpu().numpy()) # input
        ax.set_title("network input=Gauss(sig) + sig")

        ax = plt.subplot(2,3,6)
        plt.plot(t, recons_out[rand_int].cpu().detach().numpy()) # output
        ax.set_title("network output (~ng)")
        
        save_folder = 'ae-g/'
        os.makedirs(save_folder, exist_ok=True)
        save_path = os.path.join(save_folder, str(iters).zfill(7) + '.png')
        print('Saving file...{}'.format(save_path))
        plt.savefig(save_path, bbox_inches='tight')
        plt.close()
          
      iters += 1

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


[  0/30][  0/3125]	Loss: 0.0014
Saving file...ae-g/0000000.png
[  0/30][100/3125]	Loss: 0.0006
[  0/30][200/3125]	Loss: 0.0006
[  0/30][300/3125]	Loss: 0.0007
[  0/30][400/3125]	Loss: 0.0007
[  0/30][500/3125]	Loss: 0.0006
[  0/30][600/3125]	Loss: 0.0005
[  0/30][700/3125]	Loss: 0.0006
[  0/30][800/3125]	Loss: 0.0006
[  0/30][900/3125]	Loss: 0.0005
[  0/30][1000/3125]	Loss: 0.0006
[  0/30][1100/3125]	Loss: 0.0006
[  0/30][1200/3125]	Loss: 0.0006
[  0/30][1300/3125]	Loss: 0.0006
[  0/30][1400/3125]	Loss: 0.0006
[  0/30][1500/3125]	Loss: 0.0006
[  0/30][1600/3125]	Loss: 0.0005
[  0/30][1700/3125]	Loss: 0.0006
[  0/30][1800/3125]	Loss: 0.0005
[  0/30][1900/3125]	Loss: 0.0005
[  0/30][2000/3125]	Loss: 0.0005
Saving file...ae-g/0002000.png
[  0/30][2100/3125]	Loss: 0.0005
[  0/30][2200/3125]	Loss: 0.0005
[  0/30][2300/3125]	Loss: 0.0006
[  0/30][2400/3125]	Loss: 0.0006
[  0/30][2500/3125]	Loss: 0.0005
[  0/30][2600/3125]	Loss: 0.0006
[  0/30][2700/3125]	Loss: 0.0005
[  0/30][2800/3125]	Loss

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


[  1/30][ 75/3125]	Loss: 0.0005
[  1/30][175/3125]	Loss: 0.0004
[  1/30][275/3125]	Loss: 0.0005
[  1/30][375/3125]	Loss: 0.0005
[  1/30][475/3125]	Loss: 0.0005
[  1/30][575/3125]	Loss: 0.0005
[  1/30][675/3125]	Loss: 0.0005
[  1/30][775/3125]	Loss: 0.0005
[  1/30][875/3125]	Loss: 0.0005
Saving file...ae-g/0004000.png
[  1/30][975/3125]	Loss: 0.0005
[  1/30][1075/3125]	Loss: 0.0004
[  1/30][1175/3125]	Loss: 0.0005
[  1/30][1275/3125]	Loss: 0.0005
[  1/30][1375/3125]	Loss: 0.0004
[  1/30][1475/3125]	Loss: 0.0005
[  1/30][1575/3125]	Loss: 0.0005
[  1/30][1675/3125]	Loss: 0.0004
[  1/30][1775/3125]	Loss: 0.0005
[  1/30][1875/3125]	Loss: 0.0005
[  1/30][1975/3125]	Loss: 0.0004
[  1/30][2075/3125]	Loss: 0.0004
[  1/30][2175/3125]	Loss: 0.0005
[  1/30][2275/3125]	Loss: 0.0005
[  1/30][2375/3125]	Loss: 0.0004
[  1/30][2475/3125]	Loss: 0.0005
[  1/30][2575/3125]	Loss: 0.0004
[  1/30][2675/3125]	Loss: 0.0004
[  1/30][2775/3125]	Loss: 0.0004
[  1/30][2875/3125]	Loss: 0.0004
Saving file...ae-g/000

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


...

[ 27/30][ 25/3125]	Loss: 0.0001
[ 27/30][125/3125]	Loss: 0.0001
[ 27/30][225/3125]	Loss: 0.0001
[ 27/30][325/3125]	Loss: 0.0001
[ 27/30][425/3125]	Loss: 0.0001
[ 27/30][525/3125]	Loss: 0.0001
[ 27/30][625/3125]	Loss: 0.0001
[ 27/30][725/3125]	Loss: 0.0001
[ 27/30][825/3125]	Loss: 0.0001
[ 27/30][925/3125]	Loss: 0.0001
[ 27/30][1025/3125]	Loss: 0.0001
[ 27/30][1125/3125]	Loss: 0.0001
[ 27/30][1225/3125]	Loss: 0.0001
[ 27/30][1325/3125]	Loss: 0.0001
[ 27/30][1425/3125]	Loss: 0.0001
[ 27/30][1525/3125]	Loss: 0.0001
[ 27/30][1625/3125]	Loss: 0.0001
Saving file...ae-g/0086000.png
[ 27/30][1725/3125]	Loss: 0.0001
[ 27/30][1825/3125]	Loss: 0.0001
[ 27/30][1925/3125]	Loss: 0.0001
[ 27/30][2025/3125]	Loss: 0.0001
[ 27/30][2125/3125]	Loss: 0.0001
[ 27/30][2225/3125]	Loss: 0.0001
[ 27/30][2325/3125]	Loss: 0.0001
[ 27/30][2425/3125]	Loss: 0.0001
[ 27/30][2525/3125]	Loss: 0.0001
[ 27/30][2625/3125]	Loss: 0.0001
[ 27/30][2725/3125]	Loss: 0.0001
[ 27/30][2825/3125]	Loss: 0.0001
[ 27/30][2925/3125]	Lo

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


[ 28/30][  0/3125]	Loss: 0.0001
[ 28/30][100/3125]	Loss: 0.0001
[ 28/30][200/3125]	Loss: 0.0001
[ 28/30][300/3125]	Loss: 0.0001
[ 28/30][400/3125]	Loss: 0.0001
[ 28/30][500/3125]	Loss: 0.0001
Saving file...ae-g/0088000.png
[ 28/30][600/3125]	Loss: 0.0001
[ 28/30][700/3125]	Loss: 0.0001
[ 28/30][800/3125]	Loss: 0.0001
[ 28/30][900/3125]	Loss: 0.0001
[ 28/30][1000/3125]	Loss: 0.0001
[ 28/30][1100/3125]	Loss: 0.0001
[ 28/30][1200/3125]	Loss: 0.0001
[ 28/30][1300/3125]	Loss: 0.0001
[ 28/30][1400/3125]	Loss: 0.0001
[ 28/30][1500/3125]	Loss: 0.0001
[ 28/30][1600/3125]	Loss: 0.0001
[ 28/30][1700/3125]	Loss: 0.0001
[ 28/30][1800/3125]	Loss: 0.0001
[ 28/30][1900/3125]	Loss: 0.0001
[ 28/30][2000/3125]	Loss: 0.0001
[ 28/30][2100/3125]	Loss: 0.0001
[ 28/30][2200/3125]	Loss: 0.0001
[ 28/30][2300/3125]	Loss: 0.0001
[ 28/30][2400/3125]	Loss: 0.0001
[ 28/30][2500/3125]	Loss: 0.0001
Saving file...ae-g/0090000.png
[ 28/30][2600/3125]	Loss: 0.0001
[ 28/30][2700/3125]	Loss: 0.0001
[ 28/30][2800/3125]	Loss

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


[ 29/30][ 75/3125]	Loss: 0.0001
[ 29/30][175/3125]	Loss: 0.0001
[ 29/30][275/3125]	Loss: 0.0001
[ 29/30][375/3125]	Loss: 0.0001
[ 29/30][475/3125]	Loss: 0.0001
[ 29/30][575/3125]	Loss: 0.0002
[ 29/30][675/3125]	Loss: 0.0001
[ 29/30][775/3125]	Loss: 0.0001
[ 29/30][875/3125]	Loss: 0.0001
[ 29/30][975/3125]	Loss: 0.0001
[ 29/30][1075/3125]	Loss: 0.0001
[ 29/30][1175/3125]	Loss: 0.0001
[ 29/30][1275/3125]	Loss: 0.0001
[ 29/30][1375/3125]	Loss: 0.0001
Saving file...ae-g/0092000.png
[ 29/30][1475/3125]	Loss: 0.0001
[ 29/30][1575/3125]	Loss: 0.0001
[ 29/30][1675/3125]	Loss: 0.0001
[ 29/30][1775/3125]	Loss: 0.0001
[ 29/30][1875/3125]	Loss: 0.0001
[ 29/30][1975/3125]	Loss: 0.0001
[ 29/30][2075/3125]	Loss: 0.0001
[ 29/30][2175/3125]	Loss: 0.0001
[ 29/30][2275/3125]	Loss: 0.0001
[ 29/30][2375/3125]	Loss: 0.0001
[ 29/30][2475/3125]	Loss: 0.0001
[ 29/30][2575/3125]	Loss: 0.0001
[ 29/30][2675/3125]	Loss: 0.0001
[ 29/30][2775/3125]	Loss: 0.0001
[ 29/30][2875/3125]	Loss: 0.0001
[ 29/30][2975/3125]	Lo

In [12]:
from google.colab import files
#!zip -r /content/file.zip /content/ae-g
!zip -r file.zip /content/ae-g
#files.download("/content/file.zip")
files.download("file.zip")

  adding: content/ae-g/ (stored 0%)
  adding: content/ae-g/0038000.png (deflated 3%)
  adding: content/ae-g/0074000.png (deflated 3%)
  adding: content/ae-g/0004000.png (deflated 2%)
  adding: content/ae-g/0008000.png (deflated 2%)
  adding: content/ae-g/0036000.png (deflated 2%)
  adding: content/ae-g/0084000.png (deflated 3%)
  adding: content/ae-g/0064000.png (deflated 3%)
  adding: content/ae-g/0032000.png (deflated 3%)
  adding: content/ae-g/0016000.png (deflated 2%)
  adding: content/ae-g/0006000.png (deflated 2%)
  adding: content/ae-g/0012000.png (deflated 2%)
  adding: content/ae-g/0090000.png (deflated 2%)
  adding: content/ae-g/0078000.png (deflated 3%)
  adding: content/ae-g/0000000.png (deflated 2%)
  adding: content/ae-g/0068000.png (deflated 3%)
  adding: content/ae-g/0060000.png (deflated 3%)
  adding: content/ae-g/0086000.png (deflated 3%)
  adding: content/ae-g/0072000.png (deflated 3%)
  adding: content/ae-g/0026000.png (deflated 2%)
  adding: content/ae-g/0010000.pn

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>